In [18]:
import pandas as pd 
import pickle as pkl
import requests
from tqdm import tqdm

In [2]:
train = pd.read_csv('data/train.csv')
train

,id,token,entity_tag,full_mention,wiki_url
0,1,-DOCSTART- (1 EU),NaN,NaN,NaN
1,2,EU,B,EU,--NME--
2,3,rejects,NaN,NaN,NaN
3,4,German,B,German,http://en.wikipedia.org/wiki/Germany
4,5,call,NaN,NaN,NaN
...,...,...,...,...,...
218500,218501,Swansea,B,Swansea,http://en.wikipedia.org/wiki/Swansea_City_A.F.C.
218501,218502,1,NaN,NaN,NaN
218502,218503,Lincoln,B,Lincoln,http://en.wikipedia.org/wiki/Lincoln_City_F.C.
218503,218504,2,NaN,NaN,NaN


In [3]:
import pandas as pd

# Assuming 'train' is your DataFrame and 'token' is the column of interest.
# First, we will define a function to check for 'DOCSTART' in the 'token' column.
def check_docstart(row):
    # This function returns the 'id' if 'token' contains 'DOCSTART', otherwise NaN
    if pd.notnull(row['token']) and 'DOCSTART' in row['token']:
        return True
    else:
        return False # Using pandas NA to handle missing values

# Apply the function across the DataFrame.
train['docstart_id'] = train.apply(check_docstart, axis=1)

# This will create a new column 'docstart_id' where the 'id' is copied if 'token' contains 'DOCSTART', otherwise it will be NaN.


In [4]:
start_ids = train[train['docstart_id']]['id'].values
start_ids

array([     1,    491,    689,    938,   1019,   1255,   1332,   1465,
         1610,   1646,   2153,   2310,   2653,   2774,   2989,   3134,
         3534,   3752,   3859,   4024,   4142,   4269,   4370,   4517,
         4594,   4752,   4873,   5070,   5162,   5591,   5833,   5934,
         6234,   6357,   6465,   6576,   6786,   7012,   7214,   7385,
         7429,   7493,   7798,   7858,   8671,   8750,   8825,   9128,
         9399,  10206,  10423,  10744,  10862,  11818,  11916,  11973,
        12004,  12230,  12360,  12635,  12941,  13131,  13230,  13377,
        13637,  13710,  14010,  14210,  14401,  14503,  14768,  14971,
        15622,  16354,  16506,  16611,  16875,  17102,  17238,  17348,
        17615,  17806,  17987,  18060,  18150,  18281,  18367,  18548,
        18692,  18827,  18955,  19148,  19442,  19644,  19820,  20231,
        20465,  20740,  20989,  21566,  21838,  21901,  22598,  22720,
        22903,  23067,  23178,  23367,  23671,  24115,  24478,  24726,
      

In [5]:
train.iloc[491]

id                492
token            Rare
entity_tag        NaN
full_mention      NaN
wiki_url          NaN
docstart_id     False
Name: 491, dtype: object

In [6]:
test = pd.read_csv('data/test.csv')
test

,id,token,entity_tag,full_mention,wiki_url
0,0,-DOCSTART- (947testa CRICKET),NaN,NaN,NaN
1,1,CRICKET,NaN,NaN,NaN
2,2,-,NaN,NaN,NaN
3,3,LEICESTERSHIRE,B,LEICESTERSHIRE,?
4,4,TAKE,NaN,NaN,NaN
...,...,...,...,...,...
104885,104885,brother,NaN,NaN,NaN
104886,104886,",",NaN,NaN,NaN
104887,104887,Bobby,B,Bobby,?
104888,104888,.,NaN,NaN,NaN


In [7]:
def normalize_string(s):
    """
    This function takes a string as input, converts it to lower case, and replaces all spaces with underscores.
    
    Args:
    s (str): The string to normalize.
    
    Returns:
    str: The normalized string.
    """
    if isinstance(s, str):  # Check if the input is a string
        return s.lower().capitalize().replace(' ', '_')
    else:
        return s 

In [19]:
submission = test.copy(deep=True)

for idx, row in tqdm(test.iterrows(), total=test.shape[0]):
    if row['wiki_url'] == '?':
        link = 'http://en.wikipedia.org/wiki/' + normalize_string(row['token'])
        response = requests.get(link)
        if response.status_code == 200:
            submission.loc[idx, 'wiki_url'] = link
        else:
            submission.loc[idx, 'wiki_url'] = 'NOT_FOUND'
    else:
        submission.loc[idx, 'wiki_url'] = 'NOT_FOUND'
submission

  1%|█                                                                                      | 1351/104890 [00:52<1:07:31, 25.55it/s]

KeyboardInterrupt



In [13]:
submission = submission.drop(columns=['token', 'entity_tag', 'full_mention'], errors='ignore')

In [10]:
from datetime import datetime

now = datetime.now()


now = datetime.now()
timestamp_str = now.strftime("%Y:%m:%d:_%H:%M:%S") 
file_name = f"submission_{timestamp_str}.csv"
submission.to_csv(file_name, index=False)


In [11]:
pd.read_csv('data/sample_submission.csv')

,id,wiki_url
0,0,NOT_FOUND
1,1,NOT_FOUND
2,2,NOT_FOUND
3,3,PREDICTED_WIKI_LINK
4,4,NOT_FOUND
5,5,NOT_FOUND
6,6,NOT_FOUND
7,7,NOT_FOUND
8,8,NOT_FOUND
9,9,NOT_FOUND
